In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import sys
import os

# --- Google Drive'daki Proje Yolu Ayarları ---
# Google Drive'ınızdaki ana proje klasörünüzün tam yolu:
drive_proje_yolu = '/content/drive/MyDrive/jetxpredictor' # Sizin verdiğiniz yol

# Veritabanı dosyasının tam yolu
db_yolu = os.path.join(drive_proje_yolu, "jetx_data.db") # Sizin verdiğiniz yol

# Python'ın modülleri (app.py, loader.py vb.) bulabilmesi için proje yolunu sys.path'e ekle
if drive_proje_yolu not in sys.path:
    sys.path.append(drive_proje_yolu)

print(f"Proje ana yolu sys.path'e eklendi: {drive_proje_yolu}")
print(f"Kullanılacak veritabanı yolu: {db_yolu}")

# Kontrol için proje klasörünün içeriğini listeleyelim (isteğe bağlı)
print(f"\n'{drive_proje_yolu}' klasörünün içeriği:")
try:
    for item in os.listdir(drive_proje_yolu):
        print(f" - {item}")
except FileNotFoundError:
    print(f"HATA: '{drive_proje_yolu}' yolu bulunamadı. Google Drive'ı doğru bağladığınızdan ve yolun doğru olduğundan emin olun.")
except Exception as e:
    print(f"Klasör içeriği listelenirken bir hata oluştu: {e}")

print(f"\nPython'ın modül arama yolları (kontrol için): {sys.path}")

Proje ana yolu sys.path'e eklendi: /content/drive/MyDrive/jetxpredictor
Kullanılacak veritabanı yolu: /content/drive/MyDrive/jetxpredictor/jetx_data.db

'/content/drive/MyDrive/jetxpredictor' klasörünün içeriği:
 - XDonusturucu.html
 - Adsız kopyası 18.txt
 - Adsız kopyası 17.txt
 - Adsız kopyası 19.txt
 - main.py
 - models
 - data_processing
 - ensemble
 - feature_engineering
 - __pycache__
 - evaluation
 - jetx_data.db
 - .ipynb_checkpoints
 - predictor_logic.py
 - app.py

Python'ın modül arama yolları (kontrol için): ['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/drive/MyDrive/jetxpredictor']


In [7]:
!pip install numpy pandas tensorflow scikit-learn scipy fastdtw matplotlib streamlit


In [8]:
# Gerekli importlar (Hücre 2'deki sys.path ayarlaması sayesinde bulunacaklar)
# Bu importların Hücre 2'deki sys.path ayarından sonra olması önemli.
try:
    from predictor_logic import JetXPredictor # JetXPredictor sınıfınızın app.py içinde olduğunu varsayıyoruz
except ModuleNotFoundError as e:
    print(f"HATA: 'app' modülü veya içindeki 'JetXPredictor' sınıfı bulunamadı. Hata: {e}")
    # drive_proje_yolu değişkeninin Hücre 2'de doğru tanımlandığından emin olun
    print(f"Kontrol edin: Hücre 2'deki 'drive_proje_yolu' ({drive_proje_yolu if 'drive_proje_yolu' in locals() else 'tanımlı değil'}) doğru mu ve içinde 'app.py' dosyası var mı?")
    print("Eğer 'app.py' yerine başka bir .py dosyasında JetXPredictor sınıfı varsa, yukarıdaki 'from app import JetXPredictor' satırını ona göre güncelleyin.")
    raise
except ImportError as e:
    print(f"Import sırasında başka bir hata oluştu (örn: app.py içindeki bir import başarısız olmuş olabilir): {e}")
    print("Bir önceki hata mesajlarını (ModuleNotFoundError) kontrol edin, 'app.py' içindeki bir kütüphane eksik olabilir (Hücre 2'deki !pip install komutunu çalıştırdınız mı?).")
    raise

print("JetXPredictor sınıfı başarıyla import edildi.")

# 1. Tahmin ediciyi başlat (db_yolu Hücre 2'de tanımlandı)
try:
    # db_yolu değişkeninin Hücre 2'de global olarak tanımlandığını varsayıyoruz.
    # Eğer tanımlı değilse bir hata alırsınız, Hücre 2'yi çalıştırmanız gerekir.
    predictor = JetXPredictor(db_path=db_yolu) # <<--- PREDICTOR BURADA TANIMLANIYOR
    print(f"JetXPredictor '{db_yolu}' veritabanı ile başarıyla başlatıldı.")
except NameError as ne:
    print(f"HATA: 'db_yolu' tanımlı değil. Lütfen Hücre 2'yi (yol ayarlama ve kütüphane kurma) çalıştırdığınızdan emin olun. Hata: {ne}")
    raise
except Exception as e:
    print(f"JetXPredictor başlatılırken bir hata oluştu: {e}")
    raise

# 2. Modelleri yükle/eğit
print("\nVeri yükleniyor ve modeller hazırlanıyor...")
print("DİKKAT: Bu işlem, veritabanınızdaki veri miktarına ve modellerinizin karmaşıklığına bağlı olarak UZUN SÜREBİLİR.")
print("Lütfen sabırla bekleyin...")
try:
    initial_df = predictor.load_data()
    if initial_df is not None:
        print(f"Veritabanından toplam {len(initial_df)} adet veri noktası yüklendi.")
        if not initial_df.empty:
            print("Modeller mevcut verilerle başlatılıyor/eğitiliyor...")
            predictor.initialize_models(initial_df) # Bu satırda modeller eğitilecektir.
            print("Modeller başarıyla hazırlandı.")
        else:
            print("Veritabanı bulundu ancak içinde 'jetx_results' tablosunda hiç veri yok.")
            print("Sistem yeni değerler girildikçe ve yeniden eğitim yapıldıkça öğrenmeye başlayacaktır.")
    else:
        print("Kritik Hata: Veritabanından veri yüklenemedi (initial_df is None). Veritabanı yolu veya yapısı bozuk olabilir.")
        raise Exception("Veri yükleme başarısız.")
except Exception as e:
    print(f"Modeller hazırlanırken/eğitilirken bir hata oluştu: {e}")
    raise

# ----- Yeniden eğitim ve önceki tahmini güncelleme için manuel takip -----
colab_new_data_count = 0
COLAB_RETRAIN_INTERVAL = 15
colab_last_prediction_info = None
# --------------------------------------------------------------------

print("\n--- Değer Giriş Döngüsü Başlatılıyor ---")
# Değer girme ve tahmin görme döngüsü
while True:
    try:
        # PREDICTOR KONTROLÜ (Her ihtimale karşı döngü başında)
        if 'predictor' not in locals() and 'predictor' not in globals():
            print("!!!! KRİTİK HATA: 'predictor' ana döngü başında TANIMLI DEĞİL! Lütfen tüm hücreleri baştan ve sırayla çalıştırın. !!!!")
            break # Hata durumunda döngüden çık

        user_main_input = input(">>> Yeni JetX değeri girin, toplu giriş için 't', çıkmak için 'q' yazın: ").strip().lower()

        if user_main_input == 'q':
            print("Çıkılıyor...")
            break
        elif user_main_input == 't' or user_main_input == 'toplu':
            print("\n--- Toplu Değer Girişi Modu ---")
            print("Değerleri her satıra bir tane VEYA aynı satıra boşlukla ayırarak girin.")
            print("Girdiğiniz sırayla işlenecektir (ilk girdiğiniz/yapıştırdığınız değer en yeni kabul edilir).")
            print("Girişi bitirmek için boş bir satır bırakıp Enter'a basın veya sadece 'son' yazın.")

            bulk_input_lines_raw = []
            while True:
                line = input("  Değerler (veya 'son'): ")
                if not line.strip() or line.strip().lower() == 'son':
                    break
                bulk_input_lines_raw.append(line.strip())

            if not bulk_input_lines_raw:
                print("Hiç değer girilmedi. Tekli giriş moduna dönülüyor.")
                continue

            all_bulk_values_to_process_str = []
            for raw_line in bulk_input_lines_raw:
                all_bulk_values_to_process_str.extend(raw_line.split())

            if not all_bulk_values_to_process_str:
                print("Geçerli sayısal değer bulunamadı. Tekli giriş moduna dönülüyor.")
                continue

            print(f"\n{len(all_bulk_values_to_process_str)} adet toplu değer veritabanına ekleniyor (ilk girilen/yapıştırılan en yeni olarak)...")
            any_data_successfully_added_in_bulk = False

            for i, value_str in enumerate(all_bulk_values_to_process_str):
                try:
                    current_new_value = float(value_str)

                    if i == 0 and colab_last_prediction_info and 'prediction_id' in colab_last_prediction_info:
                        print(f"    -> Toplu girişten önceki tahmin (ID: {colab_last_prediction_info['prediction_id']}) sonucu {current_new_value} olarak güncelleniyor...")
                        update_success = predictor.update_result(colab_last_prediction_info['prediction_id'], current_new_value)
                        if update_success: print("       Güncelleme başarılı.")
                        else: print("       Güncelleme BAŞARISIZ OLDU.")

                    new_data_id_temp = predictor.add_new_result(current_new_value)
                    if new_data_id_temp:
                        colab_new_data_count += 1
                        any_data_successfully_added_in_bulk = True
                    else:
                        print(f"     Değer {current_new_value} eklenirken hata oluştu.")
                except ValueError:
                    print(f"Hatalı değer formatı: '{value_str}'. Bu değer atlanıyor.")
                except Exception as e:
                    print(f"Toplu değer '{value_str}' işlenirken beklenmedik hata: {e}. Bu değer atlanıyor.")

            if any_data_successfully_added_in_bulk:
                 print(f"\n--- Toplam {colab_new_data_count} yeni değer (bu toplu giriş dahil) veritabanına eklendi ---") # colab_new_data_count toplamı gösterir
            else:
                print("\nToplu girişte veritabanına hiç yeni değer eklenemedi.")
                continue

            if any_data_successfully_added_in_bulk and colab_new_data_count >= COLAB_RETRAIN_INTERVAL:
                print(f"\n-> {colab_new_data_count} yeni veri birikti. Modeller yeniden eğitiliyor (Bu işlem zaman alabilir)...")
                retrain_success = predictor.retrain_models()
                if retrain_success:
                    colab_new_data_count = 0
                    print("   Modeller yeniden eğitildi.")
                else:
                    print("   Modeller yeniden eğitilirken HATA oluştu.")
            elif any_data_successfully_added_in_bulk:
                print(f"\n{colab_new_data_count} adet yeni veri birikti ancak yeniden eğitim eşiği ({COLAB_RETRAIN_INTERVAL}) henüz aşılmadı. Modeller bu adımda yeniden EĞİTİLMEDİ.")

            if any_data_successfully_added_in_bulk:
                print("\n-> Tüm toplu girişler sonrası bir sonraki GERÇEK tur için final tahmin yapılıyor...")
                prediction_output = predictor.predict_next()
                if prediction_output:
                    print("\n========== FİNAL TAHMİN SONUCU (Toplu Giriş Sonrası) ==========")
                    predicted_val = prediction_output.get('predicted_value')
                    predicted_val_str = f"{predicted_val:.2f}" if predicted_val is not None else "N/A"
                    above_prob = prediction_output.get('above_threshold_probability')
                    above_prob_str = f"{above_prob:.2f}" if above_prob is not None else "N/A"
                    confidence = prediction_output.get('confidence_score')
                    confidence_str = f"{confidence:.2f}" if confidence is not None else "N/A"
                    prediction_id = prediction_output.get('prediction_id', "N/A")

                    print(f"  Tahmini Değer: {predicted_val_str}")
                    print(f"  1.5 Üstü Olasılığı: {above_prob_str}")
                    print(f"  Güven Skoru: {confidence_str}")
                    print(f"  Tahmin Veritabanı ID: {prediction_id}")
                    print("===============================================================\n")
                    colab_last_prediction_info = prediction_output
                else:
                    print("   Final tahmin yapılamadı.")
                    colab_last_prediction_info = None
            else:
                print("\nToplu girişte hiç veri eklenmediği için yeni bir final tahmin yapılmadı.")
            continue

        # Tekli Değer Girişi
        try:
            new_value = float(user_main_input)

            if colab_last_prediction_info and 'prediction_id' in colab_last_prediction_info:
                print(f"\n-> Önceki tahmin (ID: {colab_last_prediction_info['prediction_id']}) sonucu {new_value} olarak güncelleniyor...")
                update_success = predictor.update_result(colab_last_prediction_info['prediction_id'], new_value)
                if update_success: print("   Güncelleme başarılı.")
                else: print("   Güncelleme BAŞARISIZ OLDU.")

            print(f"\n-> Yeni değer ({new_value}) veritabanına ekleniyor...")
            new_data_id = predictor.add_new_result(new_value)
            if new_data_id:
                print(f"   Değer eklendi (Veri ID: {new_data_id}).")
                colab_new_data_count += 1
            else:
                print("   Değer eklenirken hata oluştu.")
                continue

            if colab_new_data_count >= COLAB_RETRAIN_INTERVAL:
                print(f"\n-> {colab_new_data_count} yeni veri eklendi. Modeller yeniden eğitiliyor...")
                retrain_success = predictor.retrain_models()
                if retrain_success:
                    colab_new_data_count = 0
                    print("   Modeller yeniden eğitildi.")
                else: print("   Modeller yeniden eğitilirken HATA oluştu.")

            print("\n-> Yeni tahmin yapılıyor...")
            prediction_output = predictor.predict_next()
            if prediction_output:
                print("\n========== TAHMİN SONUCU ==========")
                predicted_val = prediction_output.get('predicted_value')
                predicted_val_str = f"{predicted_val:.2f}" if predicted_val is not None else "N/A"
                above_prob = prediction_output.get('above_threshold_probability')
                above_prob_str = f"{above_prob:.2f}" if above_prob is not None else "N/A"
                confidence = prediction_output.get('confidence_score')
                confidence_str = f"{confidence:.2f}" if confidence is not None else "N/A"
                prediction_id = prediction_output.get('prediction_id', "N/A")

                print(f"  Tahmini Değer: {predicted_val_str}")
                print(f"  1.5 Üstü Olasılığı: {above_prob_str}")
                print(f"  Güven Skoru: {confidence_str}")
                print(f"  Tahmin Veritabanı ID: {prediction_id}")
                print("===================================\n")
                colab_last_prediction_info = prediction_output
            else:
                print("   Tahmin yapılamadı.")
                colab_last_prediction_info = None

        except ValueError:
            if user_main_input:
                print("Geçersiz komut veya değer. Lütfen sayısal bir değer, 't' (toplu giriş) veya 'q' (çıkış) girin.")

    except NameError as ne_outer: # Ana döngüdeki NameError'ı yakala
        print(f"Ana döngüde NameError: {ne_outer}")
        print("   'predictor' değişkeni ana döngüde tanımsız kalmış olabilir. Lütfen hücreleri doğru sırada çalıştırdığınızdan emin olun.")
        break
    except Exception as e:
        print(f"Döngü içinde beklenmedik bir genel hata oluştu: {e}")
        # İsterseniz burada döngüyü kırabilirsiniz veya bir sonraki girişe devam edebilirsiniz.
        # break

JetXPredictor sınıfı başarıyla import edildi.
JetX Tahmin Sistemi başlatılıyor...
CrashDetector başlatıldı: Geriye bakış=7, Özellikler: ['avg_last_n', 'count_very_low', 'all_above_sustained_high', 'is_decreasing_sharply']
JetXPredictor '/content/drive/MyDrive/jetxpredictor/jetx_data.db' veritabanı ile başarıyla başlatıldı.

Veri yükleniyor ve modeller hazırlanıyor...
DİKKAT: Bu işlem, veritabanınızdaki veri miktarına ve modellerinizin karmaşıklığına bağlı olarak UZUN SÜREBİLİR.
Lütfen sabırla bekleyin...
Veritabanından veriler yükleniyor: /content/drive/MyDrive/jetxpredictor/jetx_data.db
Toplam 6091 veri noktası yüklendi.
Veritabanından toplam 6091 adet veri noktası yüklendi.
Modeller mevcut verilerle başlatılıyor/eğitiliyor...
Toplam 6091 adet veri mevcut. Eğitim için son 2500 veri kullanılacak (Kayan Pencere).
Toplam 2500 veri noktası ile modeller eğitiliyor...
Alt modeller (base models) eğitiliyor...
Tümleşik Özellikli RandomForestJetXPredictor başlatıldı: min_history=201, n_estimat

KeyboardInterrupt: 